<a href="https://colab.research.google.com/github/ucfilho/raianars_adjust_RTC/blob/master/RTC_adjust_v_09_mar_08_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x):
  global Trocas,D, tempInicial, tempAtual 
  cols=len(tempInicial) 
  Ind_I=np.zeros((D,Trocas)).astype(np.int64)
  Ind_J=np.zeros((D,Trocas)).astype(np.int64)
  Tin=np.zeros((D,Trocas))
  Tout=np.zeros((D,Trocas))
  deltaT=np.zeros((D,Trocas))
  for i in range(D):
    for j in range(Trocas):
      Ind_I[i,j]=round(x[i,j])
      Ind_J[i,j]=round(x[i,j+Trocas])
      deltaT[i,j]=x[i,j+2*Trocas] 
    for jj in range(cols):
      Tin[i,jj]=tempInicial[jj]
      Tout[i,jj]=tempAtual[jj]
  return Ind_I,Ind_J,deltaT, Tin, Tout

In [0]:
#def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):
def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout):
#def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,tempInicial,tempAtual):
  global Trocas,D
  # cols=len(mCp)
  # i represent hot stream  and j represent cold stream 
  # k represent sequential heat exchanges
  cargaTermica=np.zeros((D,Trocas))
  contador=0
  cont_uso=0
  erro=np.zeros((D)) # valor erro se torna erro=1 quando ha algum erro e vai penalizar
  for ki in range(D):
    for kj in range(Trocas):
      k=kj
      i=Ind_I[ki,kj]
      j=Ind_J[ki,kj]
      '''
      print('=================')
      print(i,j)
      print('mCp[i]',mCp[i])
      print('deltaT[ki,kj]',deltaT[ki,kj])
      '''
      cargaTermica[ki,kj]=mCp[i]*deltaT[ki,kj]
      tempSaidaQ=Tin[ki,i]+deltaT[ki,kj]
      tempSaidaF=-cargaTermica[ki,kj]/(mCp[j])+Tout[ki,j]
      flag=0 # flag : indica que nao entra no teste logico
      # usei flag no lugar q ifs pq a identacao ficou melhor assim
      if(Tin[ki,i] > Tin[ki,j]): #Tquente maior Tfria: vaores atuais
        if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
          # if(contador < cols):
          flag=1
            
      if(flag>0):
        Tout[ki,i]=tempSaidaQ
        Tout[ki,j]=tempSaidaF
      else:
        erro[ki]=1


  return Tout,cargaTermica,erro

In [0]:
def Area(U,Tin,Tout,Ind_I,Ind_J,deltaT):

  global Trocas,mCp,D
  
  Area=[]

  for ki in range(D):
    Area_Particula=[]
    for kj in range(Trocas):
      i=Ind_I[ki,kj]
      j=Ind_J[ki,kj]
      '''
      print('================')
      print(i,j)
      print('mCp[i]',mCp[i])
      print('deltaT[ki,kj]',deltaT[ki,kj])
      print('ki',ki)
      print(Tout)
      print('Tout[ki,j]',Tout[ki,j])
      '''
      Carga=mCp[i]*deltaT[ki,kj]
      tempSaidaF=Tout[ki,j]
      tempSaidaQ=Tout[ki,i]
      deltaT1=Tin[ki,i]-tempSaidaF
      deltaT2=tempSaidaQ-Tin[ki,j]
      try:
        A=(deltaT1-deltaT2)
        B=np.log((deltaT1/deltaT2))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
  
      if(deltaTMediaLog==1e99):
        AreaCalc=1e99
      else:
        AreaCalc=abs(Carga/(U*deltaTMediaLog))

      if(Tout[ki,i]==Tin[ki,i]):
            AreaCalc=0

      if(Tout[ki,j]==Tin[ki,j]):
        AreaCalc=0
            
      Area_Particula.append(AreaCalc)
      
    Area.append(Area_Particula)

  return Area

In [0]:
def Utilidades(tempAlvo,Tout,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353

  
  Areas=[]

  for ki in range(D):
    areaTrocador=[]
    for j in range(cols):
      delta=tempAlvo[j]-Tout[ki,j]
      if(delta>0):#usa UQ
        cargaUQ=cargaUQ+mCp[j]*delta
        deltaT1=(TUQin-tempAlvo[j])
        deltaT2=(TUQout-Tout[ki,j])

        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99

        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(delta)/(U*deltaTMediaLog))))

      elif(delta==0):#nao usa UF e UQ
        deltaTMediaLog=0
        areaTrocador.append(0) 

      else:#usa UF
        deltaT1=(Tout[ki,j]-TUFout)
        deltaT2=(tempAlvo[j]-TUFin)     
        cargaUF=cargaUF+mCp[j]*(-delta)

        #UF -> água resfriada -> entra a 303K e sai a 353K
        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99
        
        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(-delta)/(U*deltaTMediaLog))))

    Areas.append(areaTrocador)
  
  # print(Areas)

  return Areas, cargaUF,cargaUQ 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
def CustoProcesso(Area,Utilidades,cargaUF,cargaUQ):
  global coefCustoUQ,coefCustoUF,D

  cost=np.zeros((D))

  for i in range(D):
    areaTrocadores=Area[i]
    areaUtilidades=Utilidades[i]
    costTrocadores=0
    for it in areaTrocadores:
      for wii in areaUtilidades:
        if(it<1e99):
          if(wii<1e99):
            costTrocadores=costTrocadores+300*(it**0.5+wii**0.5)
          else:
            costTrocadores=1e99
            break
          
    costUQ=coefCustoUQ*cargaUQ
    costUF=coefCustoUF*cargaUF

    if(costTrocadores==1e99):
      cost[i]=1e99
    else:
      cost[i]=costUQ+costUF+costTrocadores
      
  return cost

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  global D
  
  funr=np.zeros((D))


  cols=len(mCp)
  Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
  tempAtual,Carga,Erro=Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout)
  areaTrocadores =Area(U,Tin,Tout,Ind_I,Ind_J,deltaT)
  areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  funr=cost
 
  #print(Erro)
  #print(funr)

  for k in range(D):
    if(Erro[k]>0):
      funr[k]=1e99

  #funr = np.where(np.isnan(cost), 1e99, cost)

  return funr,Erro

In [0]:
#********************PROGRAMA PRINCIPAL*************************
D=10
Trocas=4
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
tempAtual=np.copy(tempInicial) 
#tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]

coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]





In [0]:
cols=12
x=np.zeros((D,cols))
MAX=np.array([3,2,3,2,0,1,1,1,0,0,0,0])
MIN=np.array([3,2,3,2,0,1,1,1,-100,-100,-100,-100])
for i in range(D):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

In [11]:
print(x)
print(x.shape)

[[  3.           2.           3.           2.           0.
    1.           1.           1.         -98.91065575 -88.09033333
  -42.38939718 -59.26099729]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -87.09804192 -54.73327827
  -86.468489   -71.49404696]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -58.3598571  -95.57611224
  -37.59572673 -88.00375116]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -97.78541681 -22.11737733
  -80.84719295 -24.83735223]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -14.49389734 -27.48935164
   -7.43654697 -17.34716002]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -67.9211808  -51.2747897
  -84.32350462 -14.13979669]
 [  3.           2.           3.           2.           0.
    1.      

In [12]:
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
print(Ind_I)


[[3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]]


In [13]:
print(Ind_J)


[[0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]]


In [14]:
Custo, Erro=FUN(x)
print(Custo)
print(Erro)

[1.00000000e+99 1.00000000e+99 1.00000000e+99 1.00000000e+99
 4.67315847e+05 1.00000000e+99 1.00000000e+99 1.00000000e+99
 1.00000000e+99 1.00000000e+99]
[1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


In [15]:
print(tempAtual)

[[453.         456.58409577 523.         490.61060282]
 [453.         434.0499587  468.26672173 533.        ]
 [453.         449.3935901  523.         495.40427327]
 [453.         428.21604717 498.16264777 533.        ]
 [463.870423   437.78220421 505.65283998 525.56345303]
 [453.         442.0609398  508.86020331 533.        ]
 [453.         492.8732477  496.48685399 508.24334463]
 [481.87001483 400.59400027 512.87466631 494.50664689]
 [453.         444.04398511 519.37057066 500.78539126]
 [453.         410.01009855 500.3198686  533.        ]]


In [16]:
# caso I: parecido, teste 
#Ind_x=[4,3,4,3]
#Ind_y=[1,2,2,2]
#deltaT=[-21,-75,-21,-27]
D=1
xtest=np.array([3,2,3,2,0,1,1,1,-21,-75,-21,-27]).reshape(1,12)
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x)
print(Ind_I)
Custo, Erro=FUN(xtest)
print(Custo)
print(Erro)

ValueError: ignored

In [0]:
print(xtest[0,10])
print(xtest)